In [ ]:
#COMFORT ZONE % PREDICT
#LSTM
#原文链接：https://blog.csdn.net/aliceyangxi1987/article/details/73420583

In [16]:
import numpy
import matplotlib.pyplot as plt
import pandas as pd
import os
import time
import math
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers.core import Dense, Activation, Dropout
from keras.layers import LSTM
from sklearn.metrics import mean_squared_error
%matplotlib inline

In [26]:
#load indoor Lobby dataset
path = r'D:/RPI/Rpi_courses/2020spring/Business Analytics Capstone/CoolGreenPower_materials/Capstone_shared/data/indoor/Lobby'
#record all the files' name 
files = os.listdir(path)    
#read the first file
df1 = pd.read_csv(path + '/' + files[0])

#read the next file and merge it with the previous file
for file in files:
    df2 = pd.read_csv(path + '/' + file)
    df1 = pd.concat([df1, df2], ignore_index = True, axis = 0)
    
df1 = df1.drop_duplicates()
df1.sort_values('Timestamp', inplace = True)
#df1.to_csv('lobby_indoor_comfort_DecToMar')
df = df1.copy()

In [27]:
df.rename(columns = {'Temperature_Fahrenheit':'F', 'Relative_Humidity':'H'}, inplace  = True)
df['comfort'] = 1
df.head()

,Timestamp,F,H,comfort
143940,2019-10-24 00:00,72.30,78.96,1
143941,2019-10-24 00:01,72.30,79.02,1
143942,2019-10-24 00:02,72.30,79.05,1
143943,2019-10-24 00:03,72.30,78.93,1
143944,2019-10-24 00:04,72.27,78.78,1


In [50]:
#time process-generate min hour date
def generate_timeattr(df, timeformat = '%Y-%m-%d %H:%M'):
    df['Timestamp'] = pd.to_datetime(df['Timestamp'], format = timeformat )
    df['date'] = df['Timestamp'].dt.date
    df['hour'] = df['Timestamp'].dt.hour
    df['minute'] = df['Timestamp'].dt.minute
    return df



#whether this minute is a comfort min:
def if_comfortzone(df):
    for x in range(len(df)-1):
        if df.iloc[x,1] > 78:
            df.iloc[x,3] = 0
        if df.iloc[x,1] < 68:
            df.iloc[x,3] = 0            
        if df.iloc[x,2] < 20:
            df.iloc[x,3] = 0
        if df.iloc[x,2] > 80:
            df.iloc[x,3] = 0
    return df



#calculate the comfortzone % for every hour:
def hour_comfortzone_perc(df):
    a = df['comfort'].groupby([df['date'],df['hour']]).sum()/60
    a = pd.DataFrame(a)
    a.reset_index(inplace = True)
    a['hour'] = a['hour'].astype(str).apply(lambda x: x.zfill(2)) + ':00'
    a['datetime'] = pd.to_datetime(a['date'].astype(str)+' '+a['hour'], format = '%Y-%m-%d %H:%M')
    a = a.set_index('datetime')
    a = a[['comfort']]
    return a


df = generate_timeattr(df)
df = if_comfortzone(df)

In [51]:
#df_m is the original dataset fed into the LSTM
df_m = hour_comfortzone_perc(df)   

#plot the dataset
#plt.plot(df_m)
#plt.show()

In [52]:
#df_m.to_csv('lobby_comfortzone%_DecToMar')
df_m

,comfort
datetime,
2019-10-24 00:00:00,1.000000
2019-10-24 01:00:00,0.666667
2019-10-24 02:00:00,0.416667
2019-10-24 03:00:00,0.000000
2019-10-24 04:00:00,0.283333
...,...
2020-03-13 19:00:00,1.000000
2020-03-13 20:00:00,1.000000
2020-03-13 21:00:00,1.000000


In [70]:
#load weather dataset
weather = pd.read_csv('Weather_Through_Feb10_2020.csv')
weather = weather.loc[weather['Location']=='Fort Myers, FL',:]
weather = weather.iloc[:,1:]
weather.rename(columns = {'DateTime':'Timestamp'}, inplace = True)
weather = generate_timeattr(weather, timeformat = '%m/%d/%Y %H:%M')
weather

,Timestamp,Temperature,Humidity,date,hour,minute
227411,2019-09-04 22:09:00,82.0,79.0,2019-09-04,22,9
227417,2019-09-04 22:19:00,82.0,79.0,2019-09-04,22,19
227423,2019-09-04 22:30:00,83.0,79.0,2019-09-04,22,30
227429,2019-09-04 22:40:00,83.0,79.0,2019-09-04,22,40
227435,2019-09-04 22:49:00,83.0,79.0,2019-09-04,22,49
...,...,...,...,...,...,...
365539,2020-02-12 03:42:00,69.0,98.0,2020-02-12,3,42
365545,2020-02-12 03:52:00,69.0,98.0,2020-02-12,3,52
365551,2020-02-12 04:02:00,69.0,98.0,2020-02-12,4,2
365557,2020-02-12 04:13:00,69.0,98.0,2020-02-12,4,13


In [101]:
#calculate hour average weather
weather1 = weather.groupby([weather['date'],weather['hour']]).mean()
weather1 = pd.DataFrame(weather1)
weather1.reset_index(inplace = True)
weather1['hour'] = weather1['hour'].astype(str).apply(lambda x: x.zfill(2)) + ':00'
weather1['datetime'] = pd.to_datetime(weather1['date'].astype(str)+' '+weather1['hour'], format = '%Y-%m-%d %H:%M')
weather1 = weather1.set_index('datetime')
weather1 = weather1.iloc[2:,2:4]
weather1.head(25)

,Temperature,Humidity
datetime,,
2019-09-05 00:00:00,82.666667,79.500000
2019-09-05 01:00:00,82.333333,81.000000
2019-09-05 02:00:00,82.000000,82.500000
2019-09-05 03:00:00,82.000000,83.000000
2019-09-05 04:00:00,82.000000,83.000000
2019-09-05 05:00:00,81.666667,83.666667
2019-09-05 06:00:00,81.500000,84.500000
2019-09-05 07:00:00,82.000000,84.500000
2019-09-05 08:00:00,83.833333,82.000000


In [100]:
#move weather forward: 2019-10-24 comfort will have Temperature and Humidity from 2019-10-25
weather1 = weather1.shift(periods = -24)
weather1

,Temperature,Humidity
datetime,,
2019-09-05 00:00:00,83.000000,86.666667
2019-09-05 01:00:00,83.000000,87.000000
2019-09-05 02:00:00,83.000000,87.000000
2019-09-05 03:00:00,82.833333,87.666667
2019-09-05 04:00:00,82.000000,88.666667
...,...,...
2020-02-12 00:00:00,NaN,NaN
2020-02-12 01:00:00,NaN,NaN
2020-02-12 02:00:00,NaN,NaN


In [103]:
#merge comfortzone and weather
df_cw = df_m.join(weather1)
df_cw = df_cw.dropna()
df_cw.to_csv('comfort_weatherlag.csv')